In [5]:
import numpy as np
import pandas as pd
from string import punctuation

In [25]:
def load_data():
    
    data_txt = open('data/shakespeare.txt')
    quatrain = []; volta = []; couplet = [];
    
    Lines = data_txt.readlines()
    count=0
    for i,line in enumerate(Lines):
        if any(char.isdigit() for char in line):
            for j in range(1,9):
                quatrain.append(Lines[i+j][:-1])
            for j in range(9,13):
                volta.append(Lines[i+j][:-1])
            for j in range(13,15):
                couplet.append(Lines[i+j][:-1])
    return quatrain,volta,couplet

def load_key():
    
    data_key = open('data/Syllable_dictionary.txt')
    Lines = data_key.readlines()
    
    key_list = [];
    
    for i,line in enumerate(Lines):
        split_str = line.strip(punctuation).split()
        if len(split_str) == 3:
            if len(split_str[1])==2:
                key_list.append([split_str[0],[int(split_str[2])],int(split_str[1][1])])
            elif len(split_str[2])==2:
                key_list.append([split_str[0],[int(split_str[1])],int(split_str[2][1])])
            else:
                key_list.append([split_str[0],[int(split_str[1]),int(split_str[2])],0])

        else:
            key_list.append([split_str[0],[int(split_str[1])],0])
    
    return key_list
    
    
    
def convert_to_nums(dataset,key_list):
    word_list = [i[0] for i in key_list]
    seq_list = []
    for i in range(0,len(dataset)):
        words = dataset[i].split()
        seq = []
        for j in range(0,len(words)):
            if words[j].lower() == "th'" or words[j].lower() == "t'":
                n = word_list.index(words[j].lower())
            else:
                n = word_list.index(words[j].lower().strip(punctuation))
            seq.append(n)
        seq_list.append(seq)
    
    return seq_list

def convert_to_nums_red_keylist(dataset,key_list):
    word_list = [i[0] for i in key_list]
    seq_list = []
    
    key_list_red = []
    used_inds = []
    for i in range(0,len(dataset)):
        words = dataset[i].split()
        seq = []
        for j in range(0,len(words)):
            if words[j].lower() == "th'" or words[j].lower() == "t'":
                n = word_list.index(words[j].lower())
            else:
                n = word_list.index(words[j].lower().strip(punctuation))
            if n not in used_inds:
                used_inds.append(n)
                key_list_red.append(key_list[n])
            seq.append(n)
        seq_list.append(seq)
    
    #convert sequence to 'reduced' indices
    seq_list_red = []
    for i in range(0,len(seq_list)):
        seq_list_red_cur = []
        for j in range(0,len(seq_list[i])):
            seq_list_red_cur.append(used_inds.index(seq_list[i][j]))
        seq_list_red.append(seq_list_red_cur)
    
    
    return seq_list_red,key_list_red

def generate_sequences():
    #CALL THIS TO GENERATE THE 3 TRAINING DATASETS
    #OUTPUTS:
    #seq_list_quatrain: list of sequences corresponding to lines in the first 2 quatrains
    #seq_list_volta: list of sequences corresponding to lines in the volta
    #seq_list_couplet: list of sequences corresponding to lines in the couplet
    #key_list: list of lists containing information stored in Syllable_dictionary:
    #Each list corresponds to a word, with index 0 being the word, 
    #index 1 being a list containing possible numbers of syllables, and index 2 being the number of syllables if it's
    #an end word.
    
    quatrain,volta,couplet = load_data()
    key_list = load_key()
    seq_list_quatrain = convert_to_nums(quatrain,key_list)
    seq_list_volta = convert_to_nums(volta,key_list)
    seq_list_couplet = convert_to_nums(couplet,key_list)
    return seq_list_quatrain,seq_list_volta,seq_list_couplet,key_list

def generate_sequences_reduced():
    #CALL THIS TO GENERATE THE 3 TRAINING DATASETS
    #OUTPUTS:
    #seq_lists: list with 3 elements corresponding to the 3 lists of sequences
    #key_lists: list with 3 elements corresponding to the 3 associated key lists
    
    quatrain,volta,couplet = load_data()
    key_list = load_key()
    seq_list_quatrain,key_list_quatrain = convert_to_nums_red_keylist(quatrain,key_list)
    seq_list_volta,key_list_volta = convert_to_nums_red_keylist(volta,key_list)
    seq_list_couplet,key_list_couplet = convert_to_nums_red_keylist(couplet,key_list)
    
    seq_lists = [seq_list_quatrain,seq_list_volta,seq_list_couplet]
    key_lists = [key_list_quatrain,key_list_volta,key_list_couplet]
    return seq_lists,key_lists
    
    
            
            

In [27]:
quatrain,volta,couplet = load_data()
key_list = load_key()

seq_lists,key_lists = generate_sequences_reduced()
print(seq_lists[0])


[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20], [21, 22, 23, 10, 24, 21, 25], [13, 26, 27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45], [34, 46, 34, 47, 28, 34, 48, 46, 49, 50], [51, 52, 53, 54, 55, 34, 56], [57, 58, 59, 60, 61, 34, 8, 62], [34, 63, 64, 65, 66, 67, 68, 69], [70, 71, 41, 72, 73, 74, 75, 76, 77], [78, 79, 80, 43, 81, 34, 82, 45], [43, 81, 15, 83, 74, 34, 84, 85], [28, 86, 87, 29, 30, 59, 88, 32], [89, 90, 91, 92, 57, 93, 94], [95, 61, 34, 96, 57, 97, 15, 98, 26, 99], [69, 100, 15, 19, 6, 98, 17, 101, 102], [103, 104, 105, 106, 69, 26, 107, 108], [26, 109, 110, 15, 111, 112, 113, 114], [115, 43, 100, 116, 66, 117, 103, 118, 119], [120, 15, 121, 74, 34, 122], [123, 124, 100, 125, 66, 126, 70, 71, 15, 127], [74, 21, 128, 28, 129, 130], [131, 132, 133, 109, 26, 134], [135, 34, 46, 34, 8, 136], [137, 138, 139, 140, 13, 141, 142], [57, 79, 143, 116, 144, 28, 145, 146, 147], [78, 148, 149, 133, 109, 26, 150], [15,